
<a id='whatsnew-0101'></a>

# v0.10.1 (January 22, 2013)

{{ header }}

This is a minor release from 0.10.0 and includes new features, enhancements,
and bug fixes. In particular, there is substantial new HDFStore functionality
contributed by Jeff Reback.

An undesired API breakage with functions taking the `inplace` option has been
reverted and deprecation warnings added.

## API changes

- Functions taking an `inplace` option return the calling object as before. A
  deprecation message has been added  
- Groupby aggregations Max/Min no longer exclude non-numeric data ([GH2700](https://github.com/pandas-dev/pandas/issues/2700))  
- Resampling an empty DataFrame now returns an empty DataFrame instead of
  raising an exception ([GH2640](https://github.com/pandas-dev/pandas/issues/2640))  
- The file reader will now raise an exception when NA values are found in an
  explicitly specified integer column instead of converting the column to float
  ([GH2631](https://github.com/pandas-dev/pandas/issues/2631))  
- DatetimeIndex.unique now returns a DatetimeIndex with the same name and  
- timezone instead of an array ([GH2563](https://github.com/pandas-dev/pandas/issues/2563))  

## New features

- MySQL support for database (contribution from Dan Allan)  

## HDFStore

You may need to upgrade your existing data files. Please visit the
**compatibility** section in the main docs.

You can designate (and index) certain columns that you want to be able to
perform queries on a table, by passing a list to `data_columns`

Retrieving unique values in an indexable or data column.

In [ ]:
# note that this is deprecated as of 0.14.0
# can be replicated by: store.select_column('df','index').unique()
store.unique('df', 'index')
store.unique('df', 'string')

You can now store `datetime64` in data columns

You can pass `columns` keyword to select to filter a list of the return
columns, this is equivalent to passing a
`Term('columns',list_of_columns_to_filter)`

`HDFStore` now serializes MultiIndex dataframes when appending tables.

```ipython
In [19]: index = pd.MultiIndex(levels=[['foo', 'bar', 'baz', 'qux'],
   ....:                               ['one', 'two', 'three']],
   ....:                       labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3, 3],
   ....:                               [0, 1, 2, 0, 1, 1, 2, 0, 1, 2]],
   ....:                       names=['foo', 'bar'])
   ....:

In [20]: df = pd.DataFrame(np.random.randn(10, 3), index=index,
   ....:                   columns=['A', 'B', 'C'])
   ....:

In [21]: df
Out[21]:
                  A         B         C
foo bar
foo one   -0.116619  0.295575 -1.047704
    two    1.640556  1.905836  2.772115
    three  0.088787 -1.144197 -0.633372
bar one    0.925372 -0.006438 -0.820408
    two   -0.600874 -1.039266  0.824758
baz two   -0.824095 -0.337730 -0.927764
    three -0.840123  0.248505 -0.109250
qux one    0.431977 -0.460710  0.336505
    two   -3.207595 -1.535854  0.409769
    three -0.673145 -0.741113 -0.110891

In [22]: store.append('mi', df)

In [23]: store.select('mi')
Out[23]:
                  A         B         C
foo bar
foo one   -0.116619  0.295575 -1.047704
    two    1.640556  1.905836  2.772115
    three  0.088787 -1.144197 -0.633372
bar one    0.925372 -0.006438 -0.820408
    two   -0.600874 -1.039266  0.824758
baz two   -0.824095 -0.337730 -0.927764
    three -0.840123  0.248505 -0.109250
qux one    0.431977 -0.460710  0.336505
    two   -3.207595 -1.535854  0.409769
    three -0.673145 -0.741113 -0.110891

# the levels are automatically included as data columns
In [24]: store.select('mi', "foo='bar'")
Out[24]:
                A         B         C
foo bar
bar one  0.925372 -0.006438 -0.820408
    two -0.600874 -1.039266  0.824758
```


Multi-table creation via `append_to_multiple` and selection via
`select_as_multiple` can create/select from multiple tables and return a
combined result, by using `where` on a selector table.

**Enhancements**

- `HDFStore` now can read native PyTables table format tables  
- You can pass `nan_rep = 'my_nan_rep'` to append, to change the default nan
  representation on disk (which converts to/from np.nan), this defaults to
  nan.  
- You can pass `index` to `append`. This defaults to `True`. This will
  automagically create indices on the *indexables* and *data columns* of the
  table  
- You can pass `chunksize=an integer` to `append`, to change the writing
  chunksize (default is 50000). This will significantly lower your memory usage
  on writing.  
- You can pass `expectedrows=an integer` to the first `append`, to set the
  TOTAL number of expected rows that `PyTables` will expected. This will
  optimize read/write performance.  
- `Select` now supports passing `start` and `stop` to provide selection
  space limiting in selection.  
- Greatly improved ISO8601 (e.g., yyyy-mm-dd) date parsing for file parsers ([GH2698](https://github.com/pandas-dev/pandas/issues/2698))  
- Allow `DataFrame.merge` to handle combinatorial sizes too large for 64-bit
  integer ([GH2690](https://github.com/pandas-dev/pandas/issues/2690))  
- Series now has unary negation (-series) and inversion (~series) operators ([GH2686](https://github.com/pandas-dev/pandas/issues/2686))  
- DataFrame.plot now includes a `logx` parameter to change the x-axis to log scale ([GH2327](https://github.com/pandas-dev/pandas/issues/2327))  
- Series arithmetic operators can now handle constant and ndarray input ([GH2574](https://github.com/pandas-dev/pandas/issues/2574))  
- ExcelFile now takes a `kind` argument to specify the file type ([GH2613](https://github.com/pandas-dev/pandas/issues/2613))  
- A faster implementation for Series.str methods ([GH2602](https://github.com/pandas-dev/pandas/issues/2602))  


**Bug Fixes**

- `HDFStore` tables can now store `float32` types correctly (cannot be
  mixed with `float64` however)  
- Fixed Google Analytics prefix when specifying request segment ([GH2713](https://github.com/pandas-dev/pandas/issues/2713)).  
- Function to reset Google Analytics token store so users can recover from
  improperly setup client secrets ([GH2687](https://github.com/pandas-dev/pandas/issues/2687)).  
- Fixed groupby bug resulting in segfault when passing in MultiIndex ([GH2706](https://github.com/pandas-dev/pandas/issues/2706))  
- Fixed bug where passing a Series with datetime64 values into to_datetime
  results in bogus output values ([GH2699](https://github.com/pandas-dev/pandas/issues/2699))  
- Fixed bug in `pattern in HDFStore` expressions when pattern is not a valid
  regex ([GH2694](https://github.com/pandas-dev/pandas/issues/2694))  
- Fixed performance issues while aggregating boolean data ([GH2692](https://github.com/pandas-dev/pandas/issues/2692))  
- When given a boolean mask key and a Series of new values, Series __setitem__
  will now align the incoming values with the original Series ([GH2686](https://github.com/pandas-dev/pandas/issues/2686))  
- Fixed MemoryError caused by performing counting sort on sorting MultiIndex
  levels with a very large number of combinatorial values ([GH2684](https://github.com/pandas-dev/pandas/issues/2684))  
- Fixed bug that causes plotting to fail when the index is a DatetimeIndex with
  a fixed-offset timezone ([GH2683](https://github.com/pandas-dev/pandas/issues/2683))  
- Corrected business day subtraction logic when the offset is more than 5 bdays
  and the starting date is on a weekend ([GH2680](https://github.com/pandas-dev/pandas/issues/2680))  
- Fixed C file parser behavior when the file has more columns than data
  ([GH2668](https://github.com/pandas-dev/pandas/issues/2668))  
- Fixed file reader bug that misaligned columns with data in the presence of an
  implicit column and a specified usecols value  
- DataFrames with numerical or datetime indices are now sorted prior to
  plotting ([GH2609](https://github.com/pandas-dev/pandas/issues/2609))  
- Fixed DataFrame.from_records error when passed columns, index, but empty
  records ([GH2633](https://github.com/pandas-dev/pandas/issues/2633))  
- Several bug fixed for Series operations when dtype is datetime64 ([GH2689](https://github.com/pandas-dev/pandas/issues/2689),
  [GH2629](https://github.com/pandas-dev/pandas/issues/2629), [GH2626](https://github.com/pandas-dev/pandas/issues/2626))  


See the [full release notes](index.ipynb#release) or issue tracker
on GitHub for a complete list.


<a id='whatsnew-0-10-1-contributors'></a>

## Contributors

A total of 17 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Andy Hayden +  
- Anton I. Sipos +  
- Chang She  
- Christopher Whelan  
- Damien Garaud +  
- Dan Allan +  
- Dieter Vandenbussche  
- Garrett Drapala +  
- Jay Parlar +  
- Thouis (Ray) Jones +  
- Vincent Arel-Bundock +  
- Wes McKinney  
- elpres  
- herrfz +  
- jreback  
- svaksha +  
- y-p  